### Práctica de Formatos de datos: CSV y JSON

#### Ejercicio 1[5 puntos]
Considerar el archivo __monumentos-edificios.csv__. El archivo contiene información sobre monumentos y edificios históricos de la ciudad de Madrid. 

Se pide crear funciones que realicen las siguientes operaciones:
* Leer los datos del archivo __monumentos-edificios.csv__
* __Consulta 1__: Obtener y  mostrar por pantalla por cada Distrito(columna DISTRITO) y por cada barrio(columna BARRIO) la información de los monumentos situados en esa zona. Concretamente de cada monumento se debe mostrar: nombre, calle con nombre, número y código postal, y longitud y latitud del lugar.
     
* __Consulta 2__: Se quieren conocer la distancia desde un lugar a los monumentos situados en un barrio concreto de Madrid. Para ello se crea una función que haga lo siguiente:
    * Pedir al usuario el nombre del barrio. Para ello se le debe mostrar un menú y que elija el nombre del barrio.
    * Recuperar de cada monumento en dicho barrio el número, nombre de la via y código postal.
    * Pedir al usario los datos del lugar donde se encuentra. Deberá indicar nombre de la calle, número y código postal.
    * Usar la libreria GeoPy para recuperar la latitud y longitud de los lugares indicados. Para usar GeoPy debéis colocar la carpeta que os he proporcionado al mismo nivel donde se encuentra vuestro notebook. Aquí tenéis un ejemplo de uso:

In [ ]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="Ejemplo")
location = geolocator.geocode(query={'street':"9 calle profesor jose garcía santesmases",'postalcode':'28040'}, addressdetails=True)
print(location.address)
print((location.latitude, location.longitude))

Profesor José García Santesmases, Ciudad Universitaria, Madrid, Área metropolitana de Madrid y Corredor del Henares, Comunidad de Madrid, 28001, España
(40.4532354, -3.7331054)


  * A continuación, debeis calcular la distancia entre cada monumento y el lugar indicado. Esto se puede hacer usando la siguiente función que os propociono.  

In [ ]:
from math import radians, cos, sin, asin, sqrt
def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    # Radius of earth in kilometers is 6371
    km = 6371* c
    return km
#Ejemplo de distancia entre la facultad y el Ministerio de Agricultura
haversine (-3.6906142029315534,40.408789366623914,  -3.7371139,40.4520948)

6.219153632127028

   * Mostrar por pantalla los resultados. Por ejemplo:
   
    ===================================================================
    
      Distancia desde Calle Profesor García Santesmases, 9 Madrid 28040
      
      Monumentos del barrio "Los Jerónimos"
      
      =================================================================
      
      Bolsa de Madrid: 5.49 kilometros
      
      Ministerio de Agricultura y Pesca: 6.21 kilometros
      
      ===================================================================
      
      
  * Preguntar al usuario si quiere hacer una nueva búsqueda. En caso afirmativo, se le vuelve a preguntar los mismos datos. En caso negativo se abandona la función. 

* __Consulta 3__: Generar un documento __Salida.csv__ donde se almacene la información generada en la Consulta 2 de la siguiente forma:

    Nombre Barrio, Datos Origen, Latitud origen, Longitud origen, Nombre monumento, Latitud monumento, Longitud monumento, Distancia calculada
    
Por ejemplo:
    
    Los Jerónimos, Calle Profesor José García Santesmases, 9, Madrid 28040, 40.4520948, -3.7371139, Bolsa de Madrid, 40.4158491, -3.6929779,5.49

Para leer el archivo csv usa un código como el siguiente:

In [ ]:
import csv
csvarchivo = open('monumentos-edificios.csv',encoding="utf8",errors='ignore')
entrada = csv.reader(csvarchivo, delimiter=";")
for i in entrada:
    print(i)

['PK', 'NOMBRE', 'PDF', 'CONTENT-URL', 'NOMBRE-VIA', 'CLASE-VIAL', 'TIPO-NUM', 'NUM', 'PLANTA', 'PUERTA', 'ESCALERAS', 'ORIENTACION', 'LOCALIDAD', 'PROVINCIA', 'CODIGO-POSTAL', 'BARRIO', 'DISTRITO', 'COORDENADA-X', 'COORDENADA-Y', 'LATITUD', 'LONGITUD', 'TELEFONO', 'FAX', 'EMAIL', 'TIPO', '']
['302283', 'Almacenes Rodrguez', 'https://patrimonioypaisaje.madrid.es/FrameWork/generacionPDFMonumenta/302283.pdf?idEdificio=0b08f7d9560a4510f7d9560a45102e085a0aRCRD&tipoMon=E', 'https://patrimonioypaisaje.madrid.es/sites/v/index.jsp?vgnextchannel=83bc3cb702aa4510VgnVCM1000008a4a900aRCRD&vgnextoid=0b08f7d9560a4510f7d9560a45102e085a0aRCRD', 'CABALLERO DE GRACIA', 'Calle', 'V', '3', '', '', '', '', 'MADRID', 'MADRID', '28013', 'SOL', 'CENTRO', '440537', '4474579', '40.419716000547446', '-3.700924275126349', '', '', '', '', ' ']
['11072416', 'Antigua Fbrica Martini &amp; Rossi', 'https://patrimonioypaisaje.madrid.es/FrameWork/generacionPDFMonumenta/11072416.pdf?idEdificio=aae94d07192ae610VgnVCM10000

__Consulta 1: Solución__

In [ ]:
import csv
from geopy.geocoders import Nominatim

#abrimos y leemos el archivo csv de entrada
csvarchivo = open('monumentos-edificios.csv',encoding="utf8",errors='ignore')
entrada = csv.DictReader(csvarchivo, delimiter=";")

datosOrdenados = sorted(entrada, key=lambda row:(row['DISTRITO'],row['BARRIO']), reverse=False) #datos ordenados por las columnas Distrito y Barrio por ese orden

#función que te devuelve la longitud y la latitud de una localización(formada por varios datos)
def longitudLatitud(i):
  geolocator = Nominatim(user_agent="Ejemplo")
  location = geolocator.geocode(query={'street': i['NUM'] + " " + i['CLASE-VIAL'] + " " + i['NOMBRE-VIA'],'postalcode':i['CODIGO-POSTAL']}, addressdetails=True)
  return (location.longitude, location.latitude)


In [ ]:
#seleccionamos el primer distrito y el primer barrio que vamos a manipular
distrito = datosOrdenados[0]['DISTRITO'] 
barrio = datosOrdenados[0]['BARRIO']
print("Distrito de " + distrito + " {")
print("\tBarrio de " + barrio + " {")

for i in datosOrdenados:
    try:
        coordenadas = longitudLatitud(i)
        longitud = str(coordenadas[0])
        latitud = str(coordenadas[1])
        if i['DISTRITO'] == distrito: 
            if i['BARRIO'] == barrio: #caso en el que escribimos los monumentos de un mismo barrio
                print("\t\tNombre: "+ i['NOMBRE'] + "; Calle con nombre: "+ i['NOMBRE-VIA'] + "; Numero: "+ i['NUM'] + "; Codigo postal: " + i['CODIGO-POSTAL'] + "; Longitud: " + longitud +"; Latitud: " + latitud)
            else: #caso en el que entramos en un nuevo barrio
                print("\t}")
                #actualizamos el nuevo barrio y lo mostramos por pantalla
                barrio = i['BARRIO']
                print("\tBarrio de " + barrio + " {")
                #mostramos la primera localización de dicho barrio
                print("\t\tNombre: "+ i['NOMBRE'] + "; Calle con nombre: "+ i['NOMBRE-VIA'] + "; Numero: "+ i['NUM'] + "; Codigo postal: " + i['CODIGO-POSTAL'] + "; Longitud: " + longitud +"; Latitud: " + latitud)      #falta todo lo demas
        else: #caso en el que entramos en un nuevo distrito, por lo que también cambiamos de barrio
            print("\t}")
            print("}")
            #actualizamos el nuevo distrito y el barrio
            distrito = i['DISTRITO']
            barrio = i['BARRIO']
            print("Distrito de " + distrito + " {")
            print("\tBarrio de " + barrio + " {")
            #mostramos la primera localización de dicho barrio
            print("\t\tNombre: "+ i['NOMBRE'] + "; Calle con nombre: "+ i['NOMBRE-VIA'] + "; Numero: "+ i['NUM'] + "; Codigo postal: " + i['CODIGO-POSTAL'] + "; Longitud: " + longitud +"; Latitud: " + latitud)
        print("}")
    except: #para controlar los fallos del csv al llamar a la función de la librería geopy
        print("La geolocalizacion ha fallado para un monumento.")


Distrito de ARGANZUELA {
	Barrio de ATOCHA {
		Nombre: Estacin de Atocha; Calle con nombre: EMPERADOR CARLOS V; Numero: 3; Codigo postal: 28045; Longitud: -3.693103; Latitud: 40.4073458
}
	}
	Barrio de CHOPERA {
		Nombre: Matadero y Mercado Municipal de Granados; Calle con nombre: CHOPERA; Numero: 14; Codigo postal: 28045; Longitud: -3.6978319; Latitud: 40.3920606
}
	}
	Barrio de DELICIAS {
		Nombre: Complejo El guila; Calle con nombre: RAMIREZ DE PRADO; Numero: 3; Codigo postal: 28045; Longitud: -3.6896341; Latitud: 40.3994273
}
	}
	Barrio de PALOS DE MOGUER {
		Nombre: Museo del Ferrocarril; Calle con nombre: DELICIAS; Numero: 1; Codigo postal: 28045; Longitud: -3.695615382856924; Latitud: 40.4044216
}
La geolocalizacion ha fallado para un monumento.
	}
}
Distrito de CENTRO {
	Barrio de CORTES {
		Nombre: Ateneo de Madrid; Calle con nombre: PRADO; Numero: 21; Codigo postal: 28014; Longitud: -3.6936301; Latitud: 40.4162001
}
La geolocalizacion ha fallado para un monumento.
La geolocal

__Consulta 2 y 3: Solución__

In [ ]:
barrios = str()
contador = 1 #para seleccionar el barrio
barrio = datosOrdenados[0]['BARRIO'] #primer barrio de una lista de barrios ordenados primero por distrito y luego por barrio
barrios = barrios + str(contador) + ". " + barrio + "\n"
listaBarrios = list() #lista que tendrá todos los barrios
listaBarrios.append(barrio)

for i in datosOrdenados:
    if i['BARRIO'] != barrio and i['BARRIO'] != "": #actualizamos la lista con los barrios existentes(sin repetir nombres de barrios)
        contador = contador + 1
        barrio = i['BARRIO']
        listaBarrios.append(barrio)
        barrios = barrios + str(contador) + ". " + barrio + "\n"

def funConsulta2(): #función que calculará la distancia entre un punto actual y varios monumentos de un barrio, además de que escribirá los resultados en un archivo .csv

  entrada = int(input("Escriba el numero del barrio de entre los posibles: \n" + barrios)) - 1 #el número seleccionado - 1 es la pos en la lista del barrio elegido

  print("Ha elegido el barrio " + listaBarrios[entrada] + "\n Escriba su ubicación actual.")
  #datos de la ubicación dada para el cálculo de las distancias
  datosNumero = input("Numero de la via: ")
  datosClase = input("Tipo de via: ")
  datosNombreCalle = input("Nombre de la via: ")
  datosCodigoPostal = input("Codigo postal: ")

  #latitud y longitud de la ubicación dada
  longlat = longitudLatitud({'NOMBRE-VIA' : datosNombreCalle, 'NUM' : datosNumero, 'CODIGO-POSTAL' : datosCodigoPostal, 'CLASE-VIAL' : datosClase})
  datosLong = longlat[0]
  datosLat = longlat[1]

  for i in datosOrdenados: #para recorrer todos los monumentos
      if i['BARRIO'] == listaBarrios[entrada]: #filtramos monumentos para el barrio seleccionado por el usuario
          #datos de la ubicación de un monumento
          numeroMonum = i['NUM']
          claseMonum = i['CLASE-VIAL']
          nombreMonum = i['NOMBRE-VIA']
          postalMonum = i['CODIGO-POSTAL']
          try:
              #latitud y longitud del monumento
              longlat2 = longitudLatitud(i)
              monumLong = longlat2[0]
              monumLat = longlat2[1]
              #calculamos la distancia 
              dist = haversine(datosLong, datosLat, monumLong, monumLat)
              #imprimimos el resultado
              print("El monumento " + i['NOMBRE'] + " está a una distancia de ti de: " + str(round(dist, 2)) + " km.")
              #escribimos el resultado en el archivo salida.csv
              salidaEscritor.writerow({'NOMBRE-BARRIO': listaBarrios[entrada], 'CLASE-VIAL': datosClase, 'CALLE-ORIGEN': datosNombreCalle, 'NUMERO-ORIGEN': datosNumero,
                                      'CODPOSTAL-ORIGEN': datosCodigoPostal,'LATITUD-ORIGEN': datosLat, 'LONGITUD-ORIGEN': datosLong, 'NOMBRE-MONUMENTO': i['NOMBRE'],
                                      'LATITUD-MONUMENTO': monumLat, 'LONGITUD-MONUMENTO': monumLong, 'DISTANCIA': dist})
          except: #para controlar los fallos del csv al llamar a la función de la librería geopy
              print("La geolocalizacion ha fallado para un monumento.")

In [ ]:
#abrimos el archivo csv de salida
archivoSalida=open("Salida.csv","w", newline = "")
fieldnames = ['NOMBRE-BARRIO', 'CLASE-VIAL', 'CALLE-ORIGEN', 'NUMERO-ORIGEN', 'CODPOSTAL-ORIGEN','LATITUD-ORIGEN',
              'LONGITUD-ORIGEN', 'NOMBRE-MONUMENTO', 'LATITUD-MONUMENTO', 'LONGITUD-MONUMENTO', 'DISTANCIA']
salidaEscritor=csv.DictWriter(archivoSalida, fieldnames = fieldnames)
salidaEscritor.writeheader()

funConsulta2()
condicion = input("¿Quiere realizar otra búsqueda?(s/n): ")

while condicion == "s": #si el usuario escribe s es porque quiere hacer más consultas
  funConsulta2()
  condicion = input("¿Quiere realizar otra búsqueda?(s/n): ")

archivoSalida.close()  #cerramos el archivo de salida



Escriba el numero del barrio de entre los posibles: 
1. ATOCHA
2. CHOPERA
3. DELICIAS
4. PALOS DE MOGUER
5. VISTA ALEGRE
6. CORTES
7. EMBAJADORES
8. JUSTICIA
9. PALACIO
10. SOL
11. UNIVERSIDAD
12. EL VISO
13. HISPANOAMERICA
14. PROSPERIDAD
15. ALMAGRO
16. GAZTAMBIDE
17. RIOS ROSAS
18. VENTAS
19. EL PARDO
20. CANILLAS
21. VALDEFUENTES
22. ARGUELLES
23. CASA DE CAMPO
24. CIUDAD UNIVERSITARIA
25. VINATEROS
26. JERNIMOS
27. LOS JERONIMOS
28. PACIFICO
29. CASTELLANA
30. GOYA
31. GUINDALERA
32. RECOLETOS
33. REJAS
34. PRADOLONGO
35. CASCO H.VALLECAS
2
Ha elegido el barrio CHOPERA
 Escriba su ubicación actual.
Numero de la via: 3
Tipo de via: calle
Nombre de la via: caballero de gracia
Codigo postal: 28013
El monumento Matadero y Mercado Municipal de Granados está a una distancia de ti de: 3.03 km.
¿Quiere realizar otra búsqueda?(s/n): n


#### Ejercicio 2[5 puntos]
Considerar el archivo __spacmsendasnaturaleza.json__. El archivo contiene información sobre sendas por la naturaleza de la Comunidad de Madrid. Se puede visualizar su contenido en:
https://datos.comunidad.madrid/catalogo/dataset/spacmsendasnaturaleza/resource/d37614e5-22c1-41b6-9334-66e7ee61975c?view_id=7bd0517d-26d5-4f27-a3f8-9489d3805757

Existen 224 sendas codificadas. Si se observa la estructura, las sendas se encuentran en un campo denominado "features", y cada documento de este campo es la descripción de una senda. 

Se pide crear funciones que realicen las siguientes operaciones:
* Leer los datos del archivo __spacmsendasnaturaleza.json__
* __Consulta 1__: Obtener y  mostrar por pantalla de una forma amigable un listado de las sendas. Se deberá mostrar por pantalla los campos de información: CDID, DS_NOMBRE, DS_CATEGORIA,DS_INICIO, DS_FINAL, DS_LONGITUD y DS_ENP, pero con los nombres que aparecen en el ejemplo siguiente. Así en vez de "DS_Nombre", debe aparecer "NombrePor ejemplo:

        Número de ruta: 77
        
        Nombre: "Senda Miraflores de la Sierra - La Morcuera - Bailanderos",
        
        Tipo: "Rutas por la Red de Espacios Naturales Protegidos",
        
        Inicio: "Miraflores de la Sierra",
        
        Final: "Bailanderos",
        
        Longitud: "11",
        
        Localización: "Parque Regional de la Cuenca Alta del Manzanares y Parque Nacional de la Sierra de Guadarrama",
     
* __Consulta 2__: Mostrar un menú al usuario donde aparezcan listados todas las localizaciones de las sendas. Es decir hay que recuperar los diferentes campos "DS_ENP". Pueden mostrarse enumerados. A continuación se le pedirá al usuario que indique una localización del listado, y como resultado se le mostrará por pantalla todas las sendas que se pueden realizar en esa localización. La información a mostrar es la misma que en la consulta 1. Una vez mostrada esta información, se le preguntará si quiere volver a buscar. En caso afirmativo, se repite el proceso, y en caso negativo, se abandona la función.

* __Consulta 3__: Se va a implementar una búsqueda imprecisa por palabra clave. Se le pedirá al usuario que introduzca por teclado un conjunto de palabras clave, y entonces se buscarán todas las sendas que contengan algunas de las palabras introducidas. Por cada senda se buscará en los campos de información DS_NOMBRE, DS_CATEGORIA,DS_INICIO, DS_FINAL, y  DS_ENP. Como resultado se le mostrará por pantalla todas las sendas que cumplen la condición. La información a mostrar es la misma que en la consulta 1. Una vez mostrada esta información, se le preguntará si quiere volver a buscar. En caso afirmativo, se repite el proceso, y en caso negativo, se abandona la función.

Para leer los archivos usa un código como el siguiente:

In [ ]:
import json
leer = json.loads(open('spacmsendasnaturaleza.json',encoding="utf8").read())
for i in range(224):
    print (leer["features"][i]["properties"])

__Consulta 1__

Obtener y  mostrar por pantalla de una forma amigable un listado de las sendas. Se deberá mostrar por pantalla los campos de información: CDID, DS_NOMBRE, DS_CATEGORIA,DS_INICIO, DS_FINAL, DS_LONGITUD y DS_ENP, pero con los nombres que aparecen en el ejemplo siguiente. Así en vez de "DS_Nombre", debe aparecer "NombrePor ejemplo:

        Número de ruta: 77
        
        Nombre: "Senda Miraflores de la Sierra - La Morcuera - Bailanderos",
        
        Tipo: "Rutas por la Red de Espacios Naturales Protegidos",
        
        Inicio: "Miraflores de la Sierra",
        
        Final: "Bailanderos",
        
        Longitud: "11",
        
        Localización: "Parque Regional de la Cuenca Alta del Manzanares y Parque Nacional de la Sierra de Guadarrama",
     

In [ ]:
# Recorrer el diccionario de sendas y mostrar la información
for i in range(224):
    try:
        print ("Número de ruta: ", leer["features"][i]["properties"]["CDID"], "\n")
    except KeyError:
        print ("Número de ruta: Desconocido \n")
    try:
        print ("Nombre: ", leer["features"][i]["properties"]["DS_NOMBRE"], "\n")
    except KeyError:
        print ("Nombre: Desconocido \n")
    try:
        print ("Tipo: ", leer["features"][i]["properties"]["DS_CATEGORIA"], "\n")
    except KeyError:
        print ("Tipo: Desconocido \n")
    try:
        print ("Inicio: ", leer["features"][i]["properties"]["DS_INICIO"], "\n")
    except KeyError:
        print ("Inicio: Desconocido \n")
    try:
        print ("Final: ", leer["features"][i]["properties"]["DS_FINAL"], "\n")
    except KeyError:
        print ("Final: Desconocido \n")
    try:
        print ("Longitud: ", leer["features"][i]["properties"]["DS_LONGITUD"], "\n")
    except KeyError:
        print ("Longitud: Desconocido \n")
    try:
        print ("Localizacion: ", leer["features"][i]["properties"]["DS_ENP"], "\n")
    except KeyError:
        print ("Localizacion: Desconocido \n")
    print("----------------------------------------------------------------------")


Número de ruta:  131 

Nombre:  Paseos por el Valle de El Paular. Camino de Rascafría a Oteruelo 

Tipo:  Rutas por la Red de Espacios Naturales Protegidos 

Inicio:  Puente del Perdón, Monasterio de El Paular 

Final:  Plaza de la Fragua de Oteruelo 

Longitud:  2 

Localizacion:  Parque Nacional de la Sierra de Guadarrama 

----------------------------------------------------------------------
Número de ruta:  132 

Nombre:  Paseos por el Valle de El Paular: Camino a la Cascada del Purgatorio 

Tipo:  Rutas por la Red de Espacios Naturales Protegidos 

Inicio:  Puente del Perdón, Monasterio de El Paular 

Final:  Cascada del Purgatorio 

Longitud:  6 

Localizacion:  Parque Nacional de la Sierra de Guadarrama 

----------------------------------------------------------------------
Número de ruta:  140 

Nombre:  Senda Soto Bayona: Tramo adaptado 

Tipo:  Rutas por la Red de Espacios Naturales Protegidos 

Inicio:  Aparcamiento situado al noroeste del casco urbano de Titulcia 

Final:

Número de ruta:  180 

Nombre:  Rutas los Cerros de Alcalá: Ecce-Homo 

Tipo:  Sendas Verdes de Madrid 

Inicio:  Aparcamiento de los Cerros de Alcalá 

Final:  Aparcamiento de la carretera M-300 (sentido Alcalá) frente al cementerio Jardín. 

Longitud:  6 

Localizacion: Desconocido 

----------------------------------------------------------------------
Número de ruta:  181 

Nombre:  Senda Circuito Paseo de Colmenar del Arroyo 

Tipo:  Sendas Verdes de Madrid 

Inicio:  Casco urbano de Comenar del Arroyo (entre los puentes del Caño y de la Fragua). 

Final:  Casco urbano de Comenar del Arroyo (entre los puentes del Caño y de la Fragua). 

Longitud:  4 

Localizacion: Desconocido 

----------------------------------------------------------------------
Número de ruta:  182 

Nombre:  Senda de la Dehesa de Navalmoral 

Tipo:  Sendas Verdes de Madrid 

Inicio:  Puente de la Fragua en Colmenar del Arroyo. Tomar C/ Calvario. 

Final:  El mismo que el inicio 

Longitud:  10 

Localizacion:

Final:  Buitrago de Lozoya 

Longitud:  15 

Localizacion: Desconocido 

----------------------------------------------------------------------
Número de ruta:  240 

Nombre:  Rutas por la Cañada Real Leonesa a través de sus dehesas: Ruta de Los Arribes 

Tipo:  Rutas por la Red de Vías Pecuarias 

Inicio:  Área de Interpretación del Paisaje, Vías Pecuarias, Zarzalejo 

Final:  Área de Interpretación del Paisaje, Vías Pecuarias, Zarzalejo 

Longitud:  21 

Localizacion: Desconocido 

----------------------------------------------------------------------
Número de ruta:  241 

Nombre:  Rutas por la Cañada Real Leonesa a través de sus dehesas: Ruta de la Ermita de Valmayor 

Tipo:  Rutas por la Red de Vías Pecuarias 

Inicio:  Plaza de la Iglesia (Iglesia de la Asunción en Valdemorillo). 

Final:  Plaza de la Iglesia (Iglesia de la Asunción en Valdemorillo). 

Longitud:  17 

Localizacion: Desconocido 

----------------------------------------------------------------------
Número de ruta

Inicio:  Plaza Mayor de Torrelaguna 

Final:  Plaza Mayor de Torrelaguna 

Longitud:  14 

Localizacion: Desconocido 

----------------------------------------------------------------------
Número de ruta:  275 

Nombre:  Rutas del Agua: Patones, Torrelaguna y Torremocha de Jarama: Paisaje patonero 

Tipo:  Rutas por la Red de Vías Pecuarias 

Inicio:  Plaza de la Constitución de Patones de Abajo 

Final:  Plaza de la Constitución de Patones de Abajo 

Longitud:  15 

Localizacion: Desconocido 

----------------------------------------------------------------------
Número de ruta:  276 

Nombre:  Rutas por los Robledales del Lozoya: El Puente Canto 

Tipo:  Rutas por la Red de Vías Pecuarias 

Inicio:  Lozoya 

Final:  Lozoya 

Longitud:  8 

Localizacion: Desconocido 

----------------------------------------------------------------------
Número de ruta:  277 

Nombre:  Rutas por los Robledales del Lozoya: El Arroyo del Villar 

Tipo:  Rutas por la Red de Vías Pecuarias 

Inicio:  Pla

__Consulta 2__

Mostrar un menú al usuario donde aparezcan listados todas las localizaciones de las sendas. Es decir hay que recuperar los diferentes campos "DS_ENP". Pueden mostrarse enumerados. A continuación se le pedirá al usuario que indique una localización del listado, y como resultado se le mostrará por pantalla todas las sendas que se pueden realizar en esa localización. La información a mostrar es la misma que en la consulta 1. Una vez mostrada esta información, se le preguntará si quiere volver a buscar. En caso afirmativo, se repite el proceso, y en caso negativo, se abandona la función.

In [ ]:
lista_enp = list()

for i in range(224):
    try:
        locate = leer["features"][i]["properties"]["DS_ENP"] # Recorremos el diccionario completo seleccionando el atributo ds_enp
        if locate not in lista_enp: # Guardamos las entradas en la lista si no existen, para evitar repetidas            
            lista_enp.append(locate)
    except KeyError:
        pass

entrada = 1

while entrada != 0:
  enum = 0

  for k in lista_enp: # Recorrer la lista de localizaciones distintas e imprimirlas por pantalla
    enum += 1
    print (enum, "-", k, "\n")

  entrada = int(input("Seleccione una localizacion (0->Salir): "))
  if entrada == 0:
    print("\nSaliendo...")
    break
  
  localizacion = lista_enp[entrada-1]
  print("\nRutas en", localizacion,":\n")

  for i in range(224): # Mostrar todas las rutas asociadas a la localización
      try:        
          if leer["features"][i]["properties"]["DS_ENP"] == localizacion: # Buscar por localizacion seleccionada
            print(leer["features"][i]["properties"]["DS_NOMBRE"]) # Imprimir la senda
      except KeyError:
          pass

  print("------------------------------------------------------------------------------------------------")


1 - Parque Nacional de la Sierra de Guadarrama 

2 - Parque Regional del Sureste 

3 - Parque Regional del curso medio del río Guadarrama y su entorno 

4 - Parque Regional de la Cuenca Alta del Manzanares y Parque Nacional de la Sierra de Guadarrama 

5 - Parque Regional de la Cuenca Alta del Manzanares 

Seleccione una localizacion (0->Salir): 2

Rutas en Parque Regional del Sureste :

Senda Soto Bayona: Tramo adaptado
Un paseo por El Campillo: el Agua
Un paseo por El Campillo: el Agua. Tramo de senda adaptado
Vuelta en bicicleta por los altos del Pingarrón desde San Martín de la Vega
Senda Botánica El Campillo
Paseo por las áreas recreativas de Chinchón
Paseo por las lagunas y la presa del río Henares
Ruta a pie por los altos y vaguadas de Rivas-Vaciamadrid
Senda Soto Bayona y los cortados de Titulcia
Paseo por el Soto de las Juntas
Rutas por la Sagra Madrileña: Los Cerros (Valdemoro - San Martín de la Vega - Ciempozuelos - Valdemoro)
Rutas por la Sagra Madrileña: La Vega del Jarama

__Consulta 3__

Se va a implementar una búsqueda imprecisa por palabra clave. Se le pedirá al usuario que introduzca por teclado un conjunto de palabras clave, y entonces se buscarán todas las sendas que contengan algunas de las palabras introducidas. Por cada senda se buscará en los campos de información DS_NOMBRE, DS_CATEGORIA,DS_INICIO, DS_FINAL, y  DS_ENP. Como resultado se le mostrará por pantalla todas las sendas que cumplen la condición. La información a mostrar es la misma que en la consulta 1. Una vez mostrada esta información, se le preguntará si quiere volver a buscar. En caso afirmativo, se repite el proceso, y en caso negativo, se abandona la función.

In [ ]:
# Funcion para imprimir la info de una ruta
def print_info(pos):
  try:
      print ("Número de ruta: ", leer["features"][pos]["properties"]["CDID"], "\n")
  except KeyError:
      print ("Número de ruta: Desconocido \n")
  try:
      print ("Nombre: ", leer["features"][pos]["properties"]["DS_NOMBRE"], "\n")
  except KeyError:
      print ("Nombre: Desconocido \n")
  try:
      print ("Tipo: ", leer["features"][pos]["properties"]["DS_CATEGORIA"], "\n")
  except KeyError:
      print ("Tipo: Desconocido \n")
  try:
      print ("Inicio: ", leer["features"][pos]["properties"]["DS_INICIO"], "\n")
  except KeyError:
      print ("Inicio: Desconocido \n")
  try:
      print ("Final: ", leer["features"][pos]["properties"]["DS_FINAL"], "\n")
  except KeyError:
      print ("Final: Desconocido \n")
  try:
      print ("Longitud: ", leer["features"][pos]["properties"]["DS_LONGITUD"], "\n")
  except KeyError:
      print ("Longitud: Desconocido \n")
  try:
      print ("Localizacion: ", leer["features"][pos]["properties"]["DS_ENP"], "\n")
  except KeyError:
      print ("Localizacion: Desconocido \n")
  print("----------------------------------------------------------------------")

# Recibe una lista de palabras y busca coincidencias en los atributos clave: nombre, categoria, inicio, fin, enp
def find_word(word, pos):
  match = 0  
  try:
      nombre = leer["features"][pos]["properties"]["DS_NOMBRE"]
      for k in word: # Por cada palabra de la lista buscamos coincidencias en cada uno de los atributros      
        if k in nombre: # Si existe una coincidencia, detiene la busqueda para ese atributo
          match = 1
          break
  except KeyError:
      pass
  try:
      categoria = leer["features"][pos]["properties"]["DS_CATEGORIA"]
      for k in word:      
        if k in categoria:
          match = 1
          break
  except KeyError:
      pass
  try:
      inicio = leer["features"][pos]["properties"]["DS_INICIO"]
      for k in word:
        if k in inicio:
          match = 1
          break
  except KeyError:
      pass
  try:
      final = leer["features"][pos]["properties"]["DS_FINAL"]
      for k in word:
        if k in final:
          match = 1
          break
  except KeyError:
      pass
  try:
      enp = leer["features"][pos]["properties"]["DS_ENP"]
      for k in word:
        if k in enp:
          match = 1
          break
  except KeyError:
      pass

  return match

In [ ]:
entrada = input("Introduzca palabra clave (n->Salir): ").split()
if entrada[0] == 'n':
  print("\nSaliendo...")

while entrada[0] != 'n':
  for i in range(224):
    if find_word(entrada, i): # Recorrer la lista de palabras de la entrada y buscar coincidencias en los atributos clave
      print_info(i) # Imprimir la informacion de la senda

  entrada = input("Introduzca palabras clave (n->Salir): ").split()
  if entrada[0] == 'n':
    print("\nSaliendo...")

Introduzca palabra clave (n->Salir): n

Saliendo...


#### Normas de entrega

* Fecha tope de entrega: 30/09/2021
* La entrega se realizará subiendo al campus virtual un notebook de Jupyter con la solución. El archivo tendrá como nombre FormatosI_GrupoX donde X será el número de grupo correspondiente.